In [1]:
%load_ext autoreload
%autoreload 2
from data import data_path
from images import images_path
from skfda.representation.grid import FDataGrid
from utils.utils_workflow import compute_mean_value, plot_step_fn
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle

In [6]:
last_simulation_finished_list = [1]
domain_range = (0, 1)
output_data_path = os.path.join(data_path, "output")
scenarios = os.listdir(output_data_path)
shapley_str = "shapley_{}.pkl"
r2_str = "r2_test_{}_{}.pkl"
models_considered = ["fnn", "knn", "lm"]

# Shapley value

In [8]:
for scenario, last_simulation_finished  in zip(scenarios, last_simulation_finished_list):
    for i_sim in range(last_simulation_finished + 1): 
        shapley_file_name = shapley_str.format(i_sim)
        shapley_file = os.path.join(output_data_path, scenario, shapley_file_name)
        with open(shapley_file, "rb") as file:
            data = pickle.load(file)

In [9]:
print(data)

{'intervals': [array([0.  , 0.25]), array([0.25, 0.5 ]), array([0.5 , 0.75]), array([0.75, 1.  ])], 'middle_points': [0.125, 0.375, 0.625, 0.875], 'lm': [0.4608213225417596, 0.44314481527365407, 0.01766101754819645, 0.023104928284616133], 'knn': [0.30546064671339196, 0.27086193992995383, -0.025845323994745022, -0.00025761690048353714], 'fnn': [0.45885640614459927, 0.4413011380599725, 0.0178831365066154, 0.023593449033710023], 'mRMR_r2': [0.51829226712457, 0.4656625600282573, 0.0449693044281266, 0.1654620160898374], 'mRMR_distance_correlation': [0.21778448546575038, 0.1881011405211597, 0.01416761879277536, 0.04618121372948081]}


In [ ]:
all_df = []
all_r2 = []
need_to_load_header = True
for scenario, last_simulation_finished  in zip(scenarios, last_simulation_finished_list):
    for i_sim in range(last_simulation_finished + 1): 
        shapley_file_name = shapley_str.format(i_sim)
        shapley_file = os.path.join(output_data_path, scenario, shapley_file_name)
        with open(shapley_file, "rb") as file:
            data = pickle.load(file)
        if need_to_load_header:
            intervals = [x[0] for x in data]
            header = compute_mean_value(intervals)
            need_to_load_header = False
        values = [[x[1] for x in data]]
            df = pd.DataFrame(
                data=values,
                columns=header
            )
            df["scenario"] = scenario
            df["model"] = current_model
            df["simulation"] = i_sim
            all_df.append(df)
        for current_model in models_considered:
            r2_file_name = r2_str.format(current_model, i_sim)
            r2_file = os.path.join(output_data_path, scenario, r2_file_name)
            with open(r2_file, "rb") as file:
                data_r2 = pickle.load(file)
            df_r2 = pd.DataFrame(
                data=[[scenario, current_model, data_r2]],
                columns=["scenario", "model", "r2"]
            )
            all_r2.append(df_r2)

In [ ]:
df_all = pd.concat(all_df)
df_all.head(10)

In [ ]:
df_all.isnull().values.any()

In [ ]:
df_all.shape[0]/3

In [ ]:
df_aggregated = df_all.groupby(["scenario", "model"]).mean().drop("simulation", axis="columns")

In [ ]:
df_scenario_1 = df_aggregated[df_aggregated.index.get_level_values("scenario").isin(["scenario_1"])]
df_scenario_2 = df_aggregated[df_aggregated.index.get_level_values("scenario").isin(["scenario_2"])]
scenario_1_np = df_scenario_1.to_numpy()
scenario_2_np = df_scenario_2.to_numpy()

In [ ]:
df_scenario_1.head()

In [ ]:
scenario_1_grid = FDataGrid(
    data_matrix=scenario_1_np,
    grid_points=header,
    domain_range=domain_range,
    sample_names=df_scenario_1.index.get_level_values("model"),
    coordinate_names=("Shapley value", ),
)
zzz = scenario_1_grid.plot(
    legend=True,
    group=df_scenario_1.index.get_level_values("model"),
)
plt.savefig(os.path.join(images_path, "scenario_1_shapley.eps"), format="eps")
plt.savefig(os.path.join(images_path, "scenario_1_shapley.pdf"), format="pdf")

In [ ]:
x_min = [x[0] for x in intervals]
x_max = [x[1] for x in intervals]
print(len(x_max))

In [ ]:
plot_step_fn(
    x_min=x_min,
    x_max=x_max,
    values=scenario_1_np,
    colors=['tab:blue', 'tab:orange', 'tab:green'],
    models=df_scenario_1.index.get_level_values("model").tolist(),
    domain_range=domain_range,
    x_lab="",
    y_lab="Shapley value (step function)",
    plt_h_line=True,
    plot_v_line=True,
)
plt.savefig(os.path.join(images_path, "scenario_1_step_fn.eps"), format="eps")
plt.savefig(os.path.join(images_path, "scenario_1_step_fn.pdf"), format="pdf")

In [ ]:
scenario_2_grid = FDataGrid(
    data_matrix=scenario_2_np,
    grid_points=header,
    domain_range=domain_range,
    sample_names=df_scenario_2.index.get_level_values("model"),
    coordinate_names=("Shapley value", ),
)
zzz = scenario_2_grid.plot(
    legend=True,
    group=df_scenario_2.index.get_level_values("model")
)
plt.savefig(os.path.join(images_path, "scenario_2_shapley.eps"), format="eps")
plt.savefig(os.path.join(images_path, "scenario_2_shapley.pdf"), format="pdf")

In [ ]:
plot_step_fn(
    x_min=x_min,
    x_max=x_max,
    values=scenario_2_np,
    colors=['tab:blue', 'tab:orange', 'tab:green'],
    models=list(df_scenario_2.index.get_level_values("model")),
    domain_range=domain_range,
    x_lab="",
    y_lab="Shapley value (step function)",
    plt_h_line=True,
    plot_v_line=True,
)
plt.savefig(os.path.join(images_path, "scenario_2_step_fn.eps"), format="eps")
plt.savefig(os.path.join(images_path, "scenario_2_step_fn.pdf"), format="pdf")

# Compute r^2 metrics 

In [ ]:
df_all_r2 = pd.concat(all_r2)
df_all_r2.head(10)

In [ ]:
df_all_r2.groupby(["scenario", "model"]).mean()

In [ ]:
df_all_r2.groupby(["scenario", "model"]).std()